In [1]:
import pandas as pd # in order to process the excel, tool Pandas is needed
df = pd.read_csv("data-national-organised.csv", encoding='utf-8') # read csv
df.head() # show data

,Level,Topic,Uni,Group,Year,Grade,Host,Location,Theme
0,一等奖,基于多元数据融合的列车晚点预测系统,中南大学,A,2018,13,长沙理工大学,湖南长沙,智慧与共享
1,一等奖,基于信息补偿和视认纠偏的地下互通交织段交通安全导控系统,武汉理工大学,A,2018,13,长沙理工大学,湖南长沙,智慧与共享
2,二等奖,基于蓝牙技术的共享单车规范化停车管理系统,长安大学,A,2018,13,长沙理工大学,湖南长沙,智慧与共享
3,二等奖,交叉口“盲选车道”设计方法与仿真实现,长沙理工大学,A,2018,13,长沙理工大学,湖南长沙,智慧与共享
4,二等奖,基于多传感器信息融合的智能停车系统,南京理工大学,A,2018,13,长沙理工大学,湖南长沙,智慧与共享


In [2]:

df.shape

(160, 9)

In [3]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
df["content_cutted"] = df.Topic.apply(chinese_word_cut)
df.content_cutted.head() # show data 

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kz/yfyyt20x75z3_h5vc9tvmpxh0000gn/T/jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built succesfully.


0                   基于 多元 数据 融合 的 列车 晚点 预测 系统
1    基于 信息 补偿 和 视认 纠偏 的 地下 互通 交织 段 交通安全 导 控系统
2                基于 蓝牙 技术 的 共享 单车 规范化 停车 管理系统
3                 交叉口 “ 盲选 车道 ” 设计 方法 与 仿真 实现
4                   基于 多 传感器 信息 融合 的 智能 停车 系统
Name: content_cutted, dtype: object

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 200 # set the number of key words
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [5]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 6 # set the number of topics for selected key words
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=6, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [6]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()
    
n_top_words = 15 # say there are 15 key words for each topic
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words) # show 

Topic #0:
设计 协同 研究 基于 优化 交通 公交 智能 技术 数据 系统 共享 车辆 仿真 装置
()
Topic #1:
仿真 基于 数据 研究 优化 协同 设计 系统 车辆 智能 装置 技术 公交 交通 共享
()
Topic #2:
技术 基于 设计 公交 仿真 系统 数据 协同 研究 装置 智能 交通 车辆 优化 共享
()
Topic #3:
智能 装置 基于 公交 车辆 交通 共享 技术 设计 数据 研究 仿真 协同 系统 优化
()
Topic #4:
系统 基于 设计 智能 车辆 交通 协同 研究 公交 技术 共享 数据 装置 仿真 优化
()
Topic #5:
基于 共享 数据 车辆 技术 仿真 公交 研究 智能 交通 设计 系统 装置 协同 优化
()


In [7]:

n_features = 150 # set the number of key words
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)
n_topics = 5 # set the number of topics for selected key words
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
n_top_words = 10 # say there are 15 key words for each topic
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words) # show 

Topic #0:
设计 基于 交通 研究 优化 智能 共享 协同 数据 车辆
()
Topic #1:
协同 优化 研究 公交 基于 仿真 技术 设计 系统 交通
()
Topic #2:
数据 仿真 研究 公交 共享 车辆 基于 设计 智能 优化
()
Topic #3:
基于 技术 车辆 装置 智能 共享 公交 交通 数据 仿真
()
Topic #4:
系统 基于 智能 公交 设计 共享 车辆 研究 交通 协同
()


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)

/anaconda2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [25/Dec/2018 19:23:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:23:19] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:23:19] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:23:19] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:27:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:27:48] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:27:48] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2018 19:27:48] "GET /LDAvis.js HTTP/1.1" 200 -


In [ ]:
n_features = 100 # set the number of key words
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
n_topics = 3 # set the number of topics for selected key words
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
tf = tf_vectorizer.fit_transform(df.content_cutted)
n_top_words = 4 # say there are 7 key words for each topic
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words) # show 

In [ ]:
df